# Bibliotecas

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_modelos as fmod
import funcoes_bow as fbow

import time

## Configurações

In [2]:
flag_cpu = True
if flag_cpu == True:
    # force CPU (make CPU visible)
    cpus = tf.config.experimental.list_physical_devices('CPU')
    print(cpus)
    tf.config.set_visible_devices([], 'GPU')  # hide the GPU
    tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
    tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Constantes

In [3]:
SEED = 42

# Leitura dos arquivos

In [4]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [5]:
#lista_df[0].dtypes

In [6]:
print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {lista_df[0].shape[0]}\t| {lista_df[2].shape[0]}\t|\nDesbalanceado\t| {lista_df[1].shape[0]}\t| {lista_df[3].shape[0]}\t|")

Tamanho dos Datasets:

		| Hard	| Soft	|
Balanceado	| 8400	| 8400	|
Desbalanceado	| 13290	| 13290	|


# Separando o Dataset

In [7]:
def train_test_valid(df):
    
    X = df[["titulo_1", "titulo_2"]]
    y = df["match"].to_list()
    
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.285, random_state = SEED, stratify = y_train_valid)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid


In [8]:
lista_train_test_valid = []
for df in lista_df:
    lista_train_test_valid.append(train_test_valid(df))

In [9]:
for nome, dataset in zip(arquivos, lista_train_test_valid):    
    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

# Experimentos Com Os Modelos

## Funções

## BERT, roBERTa, XLMR e ELECTRA

In [10]:
a

NameError: name 'a' is not defined

In [ ]:
for modelo in modelos:
    
    lista_df_resultado = []
    
    for nome, dataset in zip(arquivos, lista_train_test_valid):    
        
        X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

        start_time = time.time()
        (nome, historico, y_test, y_pred) = fmod.pipeline_bert(modelo, nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])
        end_time = time.time()
        runtime = end_time - start_time
        
        pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{modelo}/{nome}_historico.csv', index = False)

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, modelo, nome, runtime)

        lista_df_resultado.append(df_resultado)

## BoW Co ocorrencia

In [ ]:
lista_df_resultado[0]

,precision,recall,f1-score,support,modelo,dataset,tempo
0,0.833333,1.000000,0.909091,2100.000000,BoW,hn_balanceado,31.66017
1,0.000000,0.000000,0.000000,420.000000,BoW,hn_balanceado,31.66017
accuracy,0.833333,0.833333,0.833333,0.833333,BoW,hn_balanceado,31.66017
macro avg,0.416667,0.500000,0.454545,2520.000000,BoW,hn_balanceado,31.66017
weighted avg,0.694444,0.833333,0.757576,2520.000000,BoW,hn_balanceado,31.66017


## Similaridade de Cosseno

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

###Bag of Words###
def get_cos(dataframe):

    cosine_list = []
    
    vectors, vectorizer = fbow.vectorize_dataframe(dataframe, binario = False)
        
    for i in range(0, len(vectors), 2):
        cosine_list.append(1 - spatial.distance.cosine(vectors[i], vectors[i+1]))
    
    return cosine_list

In [12]:
a = get_cos(X_test)